In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

def load_point_cloud(filename):
    if filename.endswith('.ply'):
        return o3d.io.read_point_cloud(filename)
    elif filename.endswith('.xyz') or filename.endswith('.txt'):
        data = np.loadtxt(filename)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(data[:, :3])
        if data.shape[1] >= 6:  # If file contains RGB values
            pcd.colors = o3d.utility.Vector3dVector(data[:, 3:6]/255.0)
        return pcd
    else:
        raise ValueError("Unsupported file format")

def load_mesh(filename):
    return o3d.io.read_triangle_mesh(filename)

def visualize_geometry(geometry):
    viewer = o3d.visualization.Visualizer()
    viewer.create_window()
    viewer.add_geometry(geometry)

    opt = viewer.get_render_option()
    opt.background_color = np.asarray([0.5, 0.5, 0.5])
    opt.point_size = 1.0

    viewer.run()
    viewer.destroy_window()

def process_point_cloud(pcd):
    # Estimate normals
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=0.1, max_nn=30))

    # Remove statistical outliers
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    return cl

def create_mesh_from_point_cloud(pcd):
    # Estimate normals if they don't exist
    if not pcd.has_normals():
        pcd.estimate_normals()

    # Create mesh using Poisson surface reconstruction
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=8, width=0, scale=1.1, linear_fit=False)

    return mesh

def main():
    # Example usage
    try:
        # Try to load point cloud (replace with your file path)
        pcd = load_point_cloud("example.ply")
        print("Point cloud loaded successfully")

        # Process point cloud
        processed_pcd = process_point_cloud(pcd)
        print("Point cloud processed")

        # Visualize point cloud
        print("Visualizing point cloud...")
        visualize_geometry(processed_pcd)

        # Create and visualize mesh
        print("Creating mesh from point cloud...")
        mesh = create_mesh_from_point_cloud(processed_pcd)
        print("Visualizing mesh...")
        visualize_geometry(mesh)

    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D WARNING] Read PLY failed: unable to open file: example.ply
Point cloud loaded successfully
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
Point cloud processed
Visualizing point cloud...
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


KeyboardInterrupt: 

In [ ]:
!pip install open3d